In [2]:
import os, time, requests, pandas as pd
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import re, html
from typing import Iterable, Dict, List
import pandas as pd
import numpy as np

In [3]:
news = pd.read_excel('../data/sp_500_constituents.xlsx')
news.head(5)

,PERMNO,Company Name,Ticker,SP500 Start,SP500 End,Search Keywords
0,10104,ORACLE CORP,ORCL,"Aug. 3, 1989","Dec. 31, 2024","ORCL, Oracle, Oracle Corp"
1,10107,MICROSOFT CORP,MSFT,1994-06-07 00:00:00,"Dec. 31, 2024","MSFT, Microsoft, Microsoft Corp"
2,10138,T ROWE PRICE GROUP INC,TROW,"Oct. 13, 1999","Dec. 31, 2024","T Rowe Price, T Rowe Price Group Inc, TROW, T ..."
3,10145,HONEYWELL INTERNATIONAL INC,HON,"Dec. 31, 1925","Dec. 31, 2024","HON, Honeywell, Honeywell International Inc"
4,10516,ARCHER DANIELS MIDLAND CO,ADM,1981-07-30 00:00:00,"Dec. 31, 2024","ADM, Archer Daniels Midland, Archer Daniels Mi..."


Problem in search keywords: 1)single alphabet 2)names that have meaning: app, apple, target, now, low,...

In [4]:
TICKER_MAP = {}

for _, row in news.iterrows():
    ticker = row['Ticker']
    name = row['Search Keywords']    
    TICKER_MAP[ticker] = [t.strip() for t in name.split(',') if t.strip()]

In [5]:
TICKER_MAP

{'ORCL': ['ORCL', 'Oracle', 'Oracle Corp'],
 'MSFT': ['MSFT', 'Microsoft', 'Microsoft Corp'],
 'TROW': ['T Rowe Price', 'T Rowe Price Group Inc', 'TROW', 'T Rowe'],
 'HON': ['HON', 'Honeywell', 'Honeywell International Inc'],
 'ADM': ['ADM',
  'Archer Daniels Midland',
  'Archer Daniels Midland Co',
  'Archer Daniels'],
 'FISV': ['FISV', 'Fiserv', 'Fiserv Inc'],
 'KO': ['Coca Cola', 'Cocala', 'KO', 'Coke'],
 'CDNS': ['CDNS',
  'Cadence Design',
  'Cadence Design Systems',
  'Cadence Design Systems Inc'],
 'ED': ['Consolidated Edison', 'Consolidated Edison Inc', 'ED'],
 'XRAY': ['Dentsply Sirona', 'Dentsply Sirona Inc', 'XRAY'],
 'FAST': ['FAST', 'Fastenal', 'Fastenal Co'],
 'DTE': ['DTE Energy', 'DTE Energy Co', 'DTE'],
 'ETN': ['ETN', 'Eaton', 'Eaton Corp', 'Eaton Corp Plc'],
 'SIVB': ['SVB Financial',
  'SVB Financial Group',
  'SIVB',
  'Silicon Valley Bank'],
 'XOM': ['Exxon Mobil', 'Exxon Mobil Corp', 'XOM'],
 'MGM': ['MGM Resorts', 'MGM Resorts International', 'MGM'],
 'WM': ['WM

In [6]:
def chunk_dict(data, size=100):
    """Yields successive size-chunks from a dictionary."""
    # Convert keys to a list once
    keys = list(data.keys())
    
    # Loop through the list of keys in chunks
    for i in range(0, len(keys), size):
        # Get a slice of keys for the current chunk
        chunk_keys = keys[i:i + size]
        
        # Yield a new dictionary built from the chunk of keys
        yield {k: data[k] for k in chunk_keys}

In [8]:
API_KEY = "07307ceb-73c1-4f5d-893d-d3f0f1bc153b"
# Yvette: "0b560fd0-d530-476b-b126-d59f10dce6ab" 
#Heming: "07307ceb-73c1-4f5d-893d-d3f0f1bc153b" 
#Kaushik: "137c8f3f-b860-4cdb-bb5e-378681cccd96" 
BASE = "https://content.guardianapis.com/search"

def chunk_dict(data, size=100):
    """Yields successive size-chunks from a dictionary."""
    # Convert keys to a list once
    keys = list(data.keys())
    
    # Loop through the list of keys in chunks
    for i in range(0, len(keys), size):
        # Get a slice of keys for the current chunk
        chunk_keys = keys[i:i + size]
        
        # Yield a new dictionary built from the chunk of keys
        yield {k: data[k] for k in chunk_keys}

def _mk_query(aliases, extra=None):
    # OR the aliases, and add a light finance context to reduce false positives
    alias_q = " OR ".join([f'"{a}"' for a in aliases])
    fin_q = (
        # Core Financial Reporting
        '(earnings OR revenue OR profit OR loss OR guidance OR forecast OR "quarterly results" OR company)'
        ' OR '
        # Stock Market & Valuation
        '(stock OR shares OR "market cap" OR valuation OR "analyst rating" OR dividend OR buyback OR markets)'
        ' OR '
        # Corporate Actions & Strategy
        '(acquisition OR merger OR takeover OR IPO OR investment OR funding OR divestment)'
        ' OR '
        # Regulatory & Legal
        '("SEC filing" OR investigation OR settlement OR lawsuit)'
    )
    parts = [f"({alias_q})", fin_q]
    if extra: parts.append(extra)
    return " AND ".join(parts)

def _mk_master_query(ticker_map):
    """Builds a single, large query for all tickers combined."""
    all_company_queries = []
    
    for ticker, aliases in ticker_map.items():
        # Create a query part for each company, including aliases and the ticker
        company_q = " OR ".join([f'"{name}"' for name in aliases])
        all_company_queries.append(f"({company_q})")

    # Join all individual company queries with OR
    master_alias_q = " OR ".join(all_company_queries)

    # Use the same robust financial context query
    fin_q = (
        # Core Financial Reporting
        '(earnings OR revenue OR profit OR loss OR guidance OR forecast OR "quarterly results" OR company)'
        ' OR '
        # Stock Market & Valuation
        '(stock OR shares OR "market cap" OR valuation OR "analyst rating" OR dividend OR buyback OR markets)'
        ' OR '
        # Corporate Actions & Strategy
        '(acquisition OR merger OR takeover OR IPO OR investment OR funding OR divestment)'
        ' OR '
        # Regulatory & Legal
        '("SEC filing" OR investigation OR settlement OR lawsuit)'
    )
    

    # Combine all parts into the final master query
    parts = [f"({master_alias_q})", fin_q]
        
    return " AND ".join(parts)

def fetch_guardian_for_ticker(TICKER_MAP, start, end, page_size=100, extra_q=None, sleep=0.2):
    """Fetch Guardian articles for one ticker across all pages."""
    q = _mk_master_query(TICKER_MAP)
    page = 1
    rows = []
    sections_to_search = [
        'business',  
        'technology', 
        'politics', 
    ]
    while True:
        params = {
            "q": q,
            "from-date": start,
            "to-date": end,
            "section": '|'.join(sections_to_search),            # keep it finance-ish; remove if too strict
            "order-by": "newest",
            "page": page,
            "page-size": page_size,           # max 200
            "show-fields": "trailText,bodyText",  # summary + raw text
            "api-key": API_KEY,
        }
        r = requests.get(BASE, params=params, timeout=200)
        r.raise_for_status()
        resp = r.json()["response"]

        for it in resp["results"]:
            f = it.get("fields", {})
            rows.append({
                "id": it["id"],
                "url": it["webUrl"],
                "title": it["webTitle"],
                "pub_date": it["webPublicationDate"],
                "summary": f.get("trailText", ""),
                "body": f.get("bodyText", ""),   # plain text body
                "section": it.get("sectionName", ""),
            })

        # pagination
        if resp["currentPage"] >= resp["pages"]:
            break
        page += 1
        time.sleep(sleep)  # be polite

    return rows

# 2) Run fetch for a basket of tickers: 1y for example
#start = datetime.strptime("02/30/2025", "%m/%d/%Y").isoformat()
#end   = datetime.strptime("03/30/2025", "%m/%d/%Y").isoformat()

target_year = 2017
current_start = datetime(target_year, 1, 1)

all_processed_rows = []

while current_start.year == target_year:
    current_end = current_start + relativedelta(months=1)
    start_str = current_start.strftime('%Y-%m-%d')
    end_str = current_end.strftime('%Y-%m-%d')
    print(f"\n{'='*20} Processing Period: {start_str} to {end_str} {'='*20}")
    ticker_batches = chunk_dict(TICKER_MAP, size=30) # Split into chunks of 50
    batch_num = 0
    for ticker_batch in ticker_batches:
        batch_num += 1
        print(f"Processing batch {batch_num} with {len(ticker_batch)} tickers...")
        batch_rows = fetch_guardian_for_ticker(ticker_batch, start_str, end_str)
        all_processed_rows.extend(batch_rows)
        print(f"Batch {batch_num} completed. Total articles so far: {len(all_processed_rows)}")
        time.sleep(1)  # Optional: Pause between batches to avoid rate limits
    current_start += relativedelta(months=1)    


#all_rows = fetch_guardian_for_ticker(TICKER_MAP, start, end)
articles = pd.DataFrame(all_processed_rows).drop_duplicates(subset=["id", "url", 'title'])
month_data = articles[['url', 'title', 'pub_date', 'summary', 'body']].copy()

# 1. Define the path for your master CSV file
master_csv_path = '../data/guardian_financial_news_master.csv'

print("\n--- Updating Master CSV File ---")

try:
    # 2. Load the existing master CSV if it exists
    master_df = pd.read_csv(master_csv_path)
    print(f"Loaded {len(master_df)} articles from '{master_csv_path}'")
    
    # 3. Combine the existing data with the newly fetched articles
    combined_df = pd.concat([master_df, month_data], ignore_index=True)
    
except FileNotFoundError:
    # If the master file doesn't exist, the new data is our starting point
    print(f"Master file not found. Creating a new one at '{master_csv_path}'")
    combined_df = month_data

# 4. Remove duplicate articles to keep the master file clean
# We use the 'url' as the unique identifier for an article.
# 'keep="last"' is good practice in case article details were ever updated.
updated_master_df = combined_df.drop_duplicates(subset=['url', 'title'], keep='last')

# 5. Save the final, de-duplicated data back to the master CSV
# 'index=False' prevents pandas from writing the DataFrame index as a column.
updated_master_df.to_csv(master_csv_path, index=False)

new_articles_count = len(updated_master_df) - (len(master_df) if 'master_df' in locals() else 0)
print(f"Added {new_articles_count} new unique articles.")
print(f"Master file successfully updated. Total articles: {len(updated_master_df)}")


==================== Processing Period: 2017-01-01 to 2017-02-01 ====================
Processing batch 1 with 30 tickers...
Batch 1 completed. Total articles so far: 375
Processing batch 2 with 30 tickers...
Batch 2 completed. Total articles so far: 844
Processing batch 3 with 30 tickers...
Batch 3 completed. Total articles so far: 1204
Processing batch 4 with 30 tickers...
Batch 4 completed. Total articles so far: 1659
Processing batch 5 with 30 tickers...
Batch 5 completed. Total articles so far: 2008
Processing batch 6 with 30 tickers...
Batch 6 completed. Total articles so far: 2363
Processing batch 7 with 30 tickers...
Batch 7 completed. Total articles so far: 2712
Processing batch 8 with 30 tickers...
Batch 8 completed. Total articles so far: 3220
Processing batch 9 with 30 tickers...
Batch 9 completed. Total articles so far: 3599
Processing batch 10 with 30 tickers...
Batch 10 completed. Total articles so far: 4051
Processing batch 11 with 30 tickers...
Batch 11 completed. Tota

articles = pd.DataFrame(all_processed_rows).drop_duplicates(subset=["id", "url", 'title'])
month_data = articles[['url', 'title', 'pub_date', 'summary', 'body']].copy()

# 1. Define the path for your master CSV file
master_csv_path = '../data/guardian_financial_news_master.csv'

print("\n--- Updating Master CSV File ---")

try:
    # 2. Load the existing master CSV if it exists
    master_df = pd.read_csv(master_csv_path)
    print(f"Loaded {len(master_df)} articles from '{master_csv_path}'")
    
    # 3. Combine the existing data with the newly fetched articles
    combined_df = pd.concat([master_df, month_data], ignore_index=True)
    
except FileNotFoundError:
    # If the master file doesn't exist, the new data is our starting point
    print(f"Master file not found. Creating a new one at '{master_csv_path}'")
    combined_df = month_data

# 4. Remove duplicate articles to keep the master file clean
# We use the 'url' as the unique identifier for an article.
# 'keep="last"' is good practice in case article details were ever updated.
updated_master_df = combined_df.drop_duplicates(subset=['url', 'title'], keep='last')

# 5. Save the final, de-duplicated data back to the master CSV
# 'index=False' prevents pandas from writing the DataFrame index as a column.
updated_master_df.to_csv(master_csv_path, index=False)

new_articles_count = len(updated_master_df) - (len(master_df) if 'master_df' in locals() else 0)
print(f"Added {new_articles_count} new unique articles.")
print(f"Master file successfully updated. Total articles: {len(updated_master_df)}")

In [50]:
combined_df.drop_duplicates(subset=['title'], keep='last').shape

(2586, 5)

In [55]:
combined_df['body'].iloc[0]

'Western Australia’s Golden Outback is known for its vast, wild beauty – but beyond the salt lakes, red-rock expanses and sweeping white-sand beaches lies a rich Aboriginal cultural landscape waiting to be explored. Here, in a place of ancient stories and deep connection to country, two Aboriginal-owned tour operators offer unforgettable experiences that reveal the spirit of the land. For anyone seeking more from their journey – deeper perspectives, cultural understanding and a sense of place shaped over tens of thousands of years – these are experiences not to miss. Walk in the footsteps of ancestors Aerial view of Hellfire Bay, near Esperance. Photo credit: Tourism Western Australia. In Esperance (Kepa Kurl), Annie Dabb and her family welcome visitors to their ancestral lands via Dabungool Cultural Experiences. Named after Annie’s grandfather, Dabungool is a family-run business grounded in Nyungah culture and a strong sense of place. Each guided walk shares cultural stories tied to t

In [29]:
duplicate_rows = combined_df[combined_df.duplicated()]

In [11]:
articles['pub_date'].max()

'2025-06-27T19:01:20Z'

In [12]:
articles['pub_date'].min()

'2025-05-30T13:05:35Z'

In [18]:
articles[articles['ticker']=='A']['summary']

994     China confirms details amid efforts to end tra...
995     Chinese owner Geely looks at options that incl...
996     Rolling coverage of the latest economic and fi...
997     Judge rejects former CEO’s request to dismiss ...
998     Sportswear company to reduce manufacturing in ...
                              ...                        
1101    From Fred ‘the Shred’ Goodwin to Nigel Farage ...
1102    Sale ends state ownership of the banking group...
1103    Bank of England policymaker plays down inflati...
1104    Chairman of British Rail at a time of major ch...
1105    Joshua Schulman’s pay includes £380,000 in mov...
Name: summary, Length: 112, dtype: object

In [15]:
articles

,ticker,id,url,title,pub_date,summary,body,section
0,MSFT,business/live/2025/jun/20/uk-government-borrow...,https://www.theguardian.com/business/live/2025...,UK grocery watchdog investigating Amazon over ...,2025-06-20T14:36:29Z,Groceries Code Adjudicator calls on suppliers ...,Time to wrap up for the day (and the week! 🎉)....,Business
1,MSFT,business/live/2025/jun/04/uk-spared-50-steel-a...,https://www.theguardian.com/business/live/2025...,UK trade secretary says ‘essential steps’ happ...,2025-06-04T14:31:03Z,EU -US trade talks going in the right directio...,UK trade minister Jonathan Reynolds has said “...,Business
2,ED,business/live/2025/jun/26/uk-businesses-tax-ri...,https://www.theguardian.com/business/live/2025...,Bank of England governor warns of slowdown in ...,2025-06-26T16:11:39Z,Andrew Bailey says slack appearing in labour m...,Time to wrap up… The governor of the Bank of E...,Business
3,ED,business/live/2025/jun/25/teslas-eu-sales-slum...,https://www.theguardian.com/business/live/2025...,Eurostar passengers suffer ‘nightmare’ disrupt...,2025-06-25T15:16:27Z,Disruption between St Pancras and Gare du Nord...,Time to wrap up…. Passengers booked on Eurosta...,Business
4,ED,business/2025/jun/08/will-north-sea-oil-and-ga...,https://www.theguardian.com/business/2025/jun/...,Will the North Sea oil and gas industry be Lab...,2025-06-08T06:00:27Z,"With Nigel Farage targeting net zero, could go...",It was inevitable that Nigel Farage would take...,Business
...,...,...,...,...,...,...,...,...
1354,KKR,business/2025/jun/03/hapless-thames-water-fina...,https://www.theguardian.com/business/2025/jun/...,Hapless Thames Water is finally drinking in th...,2025-06-03T18:57:52Z,Can troubled UK utilities company find any wil...,Call yourself barbarians at the gate? Actually...,Business
1355,KKR,business/live/2025/jun/03/china-factory-activi...,https://www.theguardian.com/business/live/2025...,Bank of England governor warns Trump tariffs h...,2025-06-03T14:10:01Z,Rolling coverage of the latest economic and fi...,Time to recap. Global policymakers continue to...,Business
1356,KKR,business/2025/jun/03/sizewell-c-nuclear-power-...,https://www.theguardian.com/business/2025/jun/...,Sizewell C nuclear power plant ‘could get go-a...,2025-06-03T10:03:59Z,Keir Starmer expected to confirm result of 15-...,UK ministers could give the go-ahead to the ne...,Business
1357,KKR,business/2025/jun/03/uk-water-industry-review-...,https://www.theguardian.com/business/2025/jun/...,Water industry in England and Wales needs ‘fun...,2025-06-03T08:49:25Z,"Report finds ‘deep-rooted, systemic’ failures ...","The “deep-rooted, systemic” problems in the wa...",Business


In [134]:
# ---- Drop-in: extract sentences mentioning each company from your `articles` DF ----
def _strip_html(x: str) -> str:
    x = html.unescape(x or "")
    return re.sub(r"<[^>]+>", " ", x)

# simple sentence splitter that works well on newsy English
_SENT_SPLIT = re.compile(r'(?<!\b[A-Z])(?<=[.!?])\s+(?=[A-Z0-9])')

def _compile_alias_regex(aliases: Iterable[str]) -> re.Pattern:
    al = []
    for a in aliases:
        a = a.strip()
        if not a: 
            continue
        a = r"\s+".join(map(re.escape, a.split()))
        al.append(fr"\b{a}\b(?:'s|’s)?")
    return re.compile("|".join(al), flags=re.IGNORECASE) if al else re.compile(r"$^")

def extract_company_sentences_from_text(text: str, aliases: Iterable[str], window: int = 0, max_len: int = 500) -> List[str]:
    text = _strip_html(text)
    sents = re.split(_SENT_SPLIT, " ".join(text.split()))
    pat = _compile_alias_regex(aliases)

    hits_idx = [i for i, s in enumerate(sents) if pat.search(s)]
    chunks = []
    for i in hits_idx:
        lo, hi = max(0, i - window), min(len(sents), i + window + 1)
        chunk = " ".join(sents[lo:hi]).strip()
        chunks.append(chunk[:max_len])
    return chunks

def get_company_mentions(
    articles: pd.DataFrame,
    alias_map: Dict[str, Iterable[str]],
    text_col: str = "body",      # or "summary" / "title"
    window: int = 0
) -> pd.DataFrame:
    """
    Returns a DataFrame with one row per (ticker, article, sentence-chunk) mention.
    Expects `articles` to have columns: ['id','ticker','title','url','pub_date', text_col]
    """
    rows = []
    for tic, aliases in alias_map.items():
        subset = articles.loc[articles["ticker"] == tic]
        for _, r in subset.iterrows():
            for snip in extract_company_sentences_from_text(r.get(text_col, "") or "", aliases, window=window):
                rows.append({
                    "ticker": tic,
                    "pub_date": r.get("pub_date"),
                    "article_id": r.get("id"),
                    "title": r.get("title"),
                    "url": r.get("url"),
                    "snippet": snip
                })
    return pd.DataFrame(rows)

mentions = get_company_mentions(articles, TICKER_MAP, text_col="body", window=1)


In [170]:
dup = mentions.loc[(mentions['ticker']=='TGT')]
all_snippets = "\n\n".join(dup['snippet'])
print(all_snippets)

The stock rose more than 6% in early trading in New York today. Kenvue has been touted by analysts as a potential takeover target for Haleon, which was itself spun from British phamaceutical giant GSK three years ago. Kenvue is under pressure from activist investors to offload underperforming brands.

And you know, I have been on the side of saying maybe the balance of risks are more on the inflationary side than the disinflationary side. “I think, through time, and also as markets reprice, that probably is changing. And personally, I’m more comfortable now than I was six, nine, 12 months ago.” Last month, inflation was unchanged at 3.8%, almost double the Bank’s 2% target. Pill is one of the more hawkish policymakers at the Bank, and opposed the decision to slow the pace of its quantitative 

Pill, though, voted for another £100bn, which would have meant actively selling more bonds than the rest of the Bank policymakers were comfortable with. Today, he explains that he wanted the Bank